In [3]:
# !pip install pulp pandas

In [1]:

import pandas as pd
from datetime import datetime
from pulp import LpProblem, LpMinimize, LpVariable, lpSum, LpStatus, PULP_CBC_CMD

In [2]:
# ---- Bay clusters (Team, BayID, Capacity) ----
bays_df = pd.read_csv('bays.csv')

# ---- Sub-teams (Team, Subteam, Shift, Size) ----
subteams_df =  pd.read_csv('subteams.csv')


In [3]:


DAYS = ["Mon","Tue","Wed","Thu","Fri"]

# ================================================================
# Helpers
# ================================================================
def parse_shift_minutes(shift: str):
    a, b = shift.split("-")
    fmt = "%H:%M"
    sa = datetime.strptime(a, fmt).hour*60 + datetime.strptime(a, fmt).minute
    sb = datetime.strptime(b, fmt).hour*60 + datetime.strptime(b, fmt).minute
    if sb <= sa:  # overnight
        sb += 24*60
    return sa, sb

def is_night(interval, i, cutoff_start=18*60, cutoff_end=22*60) -> bool:
    s, e = interval[i]["StartMin"], interval[i]["EndMin"]
    return (e > 24*60) or (s >= cutoff_start) or (e > cutoff_end)

# ================================================================
# 1) Split oversized sub-teams using TEAM TOTAL capacity only
# ================================================================
def split_by_team_total(bays_df: pd.DataFrame, subteams_df: pd.DataFrame):
    bay_capacity = bays_df.set_index("BayID")["Capacity"].to_dict()
    team_to_bays = bays_df.groupby("Team")["BayID"].apply(list).to_dict()
    team_total_cap = {t: sum(bay_capacity[b] for b in team_to_bays.get(t, [])) for t in team_to_bays}

    st = subteams_df.copy()
    if "StartMin" not in st.columns or "EndMin" not in st.columns:
        st["StartMin"], st["EndMin"] = zip(*st["Shift"].map(parse_shift_minutes))

    rows, mapping = [], []
    for _, r in st.iterrows():
        team, sub, size, shift = r["Team"], r["Subteam"], int(r["Size"]), r["Shift"]
        smin, emin = int(r["StartMin"]), int(r["EndMin"])
        cap = team_total_cap.get(team, 0)

        if cap == 0 or size <= cap:
            mname = f"{sub}#1"
            rows.append((team, mname, shift, size, smin, emin, sub))
            mapping.append((mname, sub, team, size))
        else:
            full_chunks, rem = divmod(size, cap)
            for j in range(1, full_chunks+1):
                mname = f"{sub}#{j}"
                rows.append((team, mname, shift, cap, smin, emin, sub))
                mapping.append((mname, sub, team, cap))
            if rem > 0:
                mname = f"{sub}#{full_chunks+1}"
                rows.append((team, mname, shift, rem, smin, emin, sub))
                mapping.append((mname, sub, team, rem))

    micro_df = pd.DataFrame(rows, columns=["Team","SubteamMicro","Shift","Size","StartMin","EndMin","ParentSubteam"])
    mapping_df = pd.DataFrame(mapping, columns=["SubteamMicro","ParentSubteam","Team","MicroSize"])
    return micro_df, mapping_df

# ================================================================
# 2) Automatic weight calculators for single-stage lexicographic
# ================================================================
def compute_safe_dominance_weights(subs, DAYS, bay_capacity, min_days_required, allow_borrow):
    """
    Build weights so each higher-priority metric dominates the combined worst-case
    of all lower-priority metrics (strict lexicographic mimic in ONE solve).
    Priorities (highest → lowest):  MissedDays  >  SplitDays  >  NightBays  > BorrowEvents
    """
    # Worst-case bounds (safe & simple)
    M_max = len(subs) * min_days_required                # everyone misses all required days
    S_max = len(subs) * len(DAYS)                        # every sub splits every day
    W_max = len(bay_capacity) * len(DAYS)                # every bay used at night every day
    R_max = (len(subs) * len(DAYS) * max(0, len(bay_capacity)-1)) if allow_borrow else 0  # loose upper bound

    # Choose the SMALLEST numbers that satisfy dominance (to reduce numeric issues)
    delta = (1.0 if allow_borrow else 0.0)               # weight for borrowing (0 if not allowed)
    beta  = (R_max + 1) * delta if allow_borrow else 1.0 # > delta*R_max
    alpha = (W_max * beta + R_max * delta + 1.0)         # > beta*W_max + delta*R_max
    gamma = (S_max * alpha + W_max * beta + R_max * delta + 1.0)  # > alpha*S_max + beta*W_max + delta*R_max

    return dict(gamma=gamma, alpha=alpha, beta=beta, delta=delta,
                M_max=M_max, S_max=S_max, W_max=W_max, R_max=R_max)

def compute_normalized_epsilon_weights(subs, DAYS, bay_capacity, min_days_required, allow_borrow):
    """
    Normalize each metric to ~[0,1], then apply a tiny epsilon ladder to enforce priority.
    Numerically friendlier; very fast in practice.
    """
    M_max = max(1, len(subs) * min_days_required)
    S_max = max(1, len(subs) * len(DAYS))
    W_max = max(1, len(bay_capacity) * len(DAYS))
    R_max = max(1, len(subs) * len(DAYS) * max(0, len(bay_capacity)-1)) if allow_borrow else 1

    eps = 1e-3  # tiny separation to break ties deterministically
    # Priority weights (MissedDays > Splits > NightBays > Borrow)
    Gamma = 1.0
    A = 1.0 - eps
    B = 1.0 - 2*eps
    Delta = (1.0 - 3*eps) if allow_borrow else 0.0

    return dict(Gamma=Gamma, A=A, B=B, Delta=Delta,
                M_max=M_max, S_max=S_max, W_max=W_max, R_max=R_max)

# ================================================================
# 3) MILP on micro-teams with AUTO weights (Strategy A in 1 solve)
# ================================================================
def solve_microteams_weighted(
    bays_df, micro_df,
    min_days_required=3,
    make_min_days_soft=True,      # keep True to let the objective act on MissedDays
    allow_borrow=False,
    slot_minutes=30,
    priority_mode="dominance",    # "dominance" or "normalized"
    verbose=True
):
    """
    Priorities (high→low):
      1) Minimize MissedDays (Σ miss[i])        — policy ≥ min_days_required
      2) Minimize SplitDays (Σ z[i,d])          — keep micro-teams together per day
      3) Minimize NightBays (Σ w[b,d])          — cluster night usage
      4) Minimize BorrowEvents (Σ x on non-owner bays) — if borrowing enabled

    'priority_mode':
      - "dominance": safe Big-M style weights computed automatically
      - "normalized": normalized terms + epsilon ladder (stable & fast)
    """

    # ---- time slots ----
    all_start = int(micro_df["StartMin"].min())
    all_end   = int(micro_df["EndMin"].max())
    TIME_SLOTS = list(range(all_start, all_end, slot_minutes))

    # ---- lookups ----
    bay_capacity = bays_df.set_index("BayID")["Capacity"].to_dict()
    bay_team     = bays_df.set_index("BayID")["Team"].to_dict()
    team_to_bays = bays_df.groupby("Team")["BayID"].apply(list).to_dict()

    subs = micro_df["SubteamMicro"].tolist()
    sub_team = micro_df.set_index("SubteamMicro")["Team"].to_dict()
    sub_size = micro_df.set_index("SubteamMicro")["Size"].to_dict()
    interval = micro_df.set_index("SubteamMicro")[["StartMin","EndMin"]].to_dict(orient="index")

    # Allowed bays
    all_bays = list(bay_capacity.keys())
    allowed_bays = {i: (all_bays if allow_borrow else team_to_bays.get(sub_team[i], [])) for i in subs}

    # ---- build MILP ----
    m = LpProblem("SeatAllocation_MicroTeams_AutoWeights", LpMinimize)

    # Vars
    a = {(i,d,b): LpVariable(f"a_{i}_{d}_{b}", lowBound=0, cat="Integer")
         for i in subs for d in DAYS for b in allowed_bays[i]}
    x = {(i,d,b): LpVariable(f"x_{i}_{d}_{b}", lowBound=0, upBound=1, cat="Binary")
         for i in subs for d in DAYS for b in allowed_bays[i]}
    y = {(i,d):    LpVariable(f"y_{i}_{d}",    lowBound=0, upBound=1, cat="Binary")
         for i in subs for d in DAYS}
    z = {(i,d):    LpVariable(f"z_{i}_{d}",    lowBound=0, upBound=1, cat="Binary")
         for i in subs for d in DAYS}
    w = {(b,d):    LpVariable(f"w_{b}_{d}",    lowBound=0, upBound=1, cat="Binary")
         for b in bay_capacity for d in DAYS}
    # Missed-days slack (needed for weighted priority on attendance)
    if not make_min_days_soft:
        raise ValueError("For weighted lexicographic, set make_min_days_soft=True so 'miss' exists.")
    miss = {i: LpVariable(f"miss_{i}", lowBound=0, cat="Integer") for i in subs}

    def covers(i, t): return interval[i]["StartMin"] <= t < interval[i]["EndMin"]

    # Min-days (soft): y + miss ≥ min_days_required
    for i in subs:
        m += lpSum(y[(i,d)] for d in DAYS) + miss[i] >= min_days_required

    # Full micro present if y=1 + split structure
    for i in subs:
        sz = sub_size[i]
        Mi = max(0, len(allowed_bays[i]) - 1)
        for d in DAYS:
            m += lpSum(a[(i,d,b)] for b in allowed_bays[i]) == sz * y[(i,d)]
            m += lpSum(x[(i,d,b)] for b in allowed_bays[i]) <= 1 + Mi * z[(i,d)]
            for b in allowed_bays[i]:
                m += a[(i,d,b)] <= sz * x[(i,d,b)]            # cannot assign more than team size
                m += a[(i,d,b)] <= bay_capacity[b] * x[(i,d,b)]  # cannot exceed bay capacity

    # Bay capacity per time slot
    for b, cap in bay_capacity.items():
        for d in DAYS:
            for t in TIME_SLOTS:
                terms = [a[(i,d,b)] for i in subs if (b in allowed_bays[i]) and covers(i,t)]
                if terms:
                    m += lpSum(terms) <= cap

    # Night clustering indicator
    night_subs = [i for i in subs if is_night(interval, i)]
    for b in bay_capacity:
        for d in DAYS:
            for i in night_subs:
                if b in allowed_bays[i]:
                    m += w[(b,d)] >= x[(i,d,b)]

    # ------------------------------
    # Objective with AUTO weights
    # ------------------------------
    if priority_mode == "dominance":
        W = compute_safe_dominance_weights(subs, DAYS, bay_capacity, min_days_required, allow_borrow)
        # For insight, you can print W once:
        # print("Auto weights (dominance):", {k: round(v,2) for k,v in W.items() if k in ("gamma","alpha","beta","delta")})

        # Σ miss (M), Σ z (S), Σ w (Wn), Σ borrowed_x (R)
        M_term = lpSum(miss[i] for i in subs)
        S_term = lpSum(z[(i,d)] for i in subs for d in DAYS)
        Wn_term = lpSum(w[(b,d)] for b in bay_capacity for d in DAYS)

        # Borrow events only if allowed
        if allow_borrow:
            R_terms = [x[(i,d,b)] for i in subs for d in DAYS for b in allowed_bays[i]
                       if bay_team[b] != sub_team[i]]
            R_term = lpSum(R_terms) if R_terms else 0
        else:
            R_term = 0

        obj = (
            W["gamma"] * M_term +
            W["alpha"] * S_term +
            W["beta"]  * Wn_term +
            W["delta"] * R_term
        )

    elif priority_mode == "normalized":
        W = compute_normalized_epsilon_weights(subs, DAYS, bay_capacity, min_days_required, allow_borrow)

        M_term = (lpSum(miss[i] for i in subs)) / W["M_max"]
        S_term = (lpSum(z[(i,d)] for i in subs for d in DAYS)) / W["S_max"]
        Wn_term = (lpSum(w[(b,d)] for b in bay_capacity for d in DAYS)) / W["W_max"]

        if allow_borrow:
            R_terms = [x[(i,d,b)] for i in subs for d in DAYS for b in allowed_bays[i]
                       if bay_team[b] != sub_team[i]]
            R_term = (lpSum(R_terms) / W["R_max"]) if R_terms else 0
        else:
            R_term = 0

        # Priority ladder: Gamma > A > B > Delta via tiny eps steps inside the constants
        obj = (
            W["Gamma"] * M_term +
            W["A"]     * S_term +
            W["B"]     * Wn_term +
            W["Delta"] * R_term
        )

    else:
        raise ValueError("priority_mode must be 'dominance' or 'normalized'")

    m += obj

    # Solve
    status = m.solve(PULP_CBC_CMD(msg=verbose))
    print("Solver status:", LpStatus[status])

    # Schedule output
    rows = []
    for i in subs:
        row = {"SubteamMicro": i}
        for d in DAYS:
            if y[(i,d)].value() < 0.5:
                row[d] = "Remote"
            else:
                parts = []
                for b in allowed_bays[i]:
                    v = a[(i,d,b)].value()
                    if v and v > 0.5:
                        parts.append(f"{b}({int(round(v))})")
                row[d] = " + ".join(parts) if parts else "Remote"
        row["MissedDays"] = int(round(miss[i].value()))
        rows.append(row)
    schedule_micro_df = pd.DataFrame(rows).set_index("SubteamMicro")
    return schedule_micro_df

# ================================================================
# 4) Orchestrator: split + auto-weighted solve
# ================================================================
def run_team_total_pipeline_weighted(
    bays_df, subteams_df,
    min_days_required=3,
    make_min_days_soft=True,
    allow_borrow=False,
    slot_minutes=30,
    priority_mode="dominance",   # "dominance" (strict mimic) or "normalized" (stable)
    verbose=True
):
    # Step 1: split by team total
    st = subteams_df.copy()
    if "StartMin" not in st.columns or "EndMin" not in st.columns:
        st["StartMin"], st["EndMin"] = zip(*st["Shift"].map(parse_shift_minutes))
    micro_df, mapping_df = split_by_team_total(bays_df, st)

    # Step 2: solve with automatic weights
    schedule_micro_df = solve_microteams_weighted(
        bays_df, micro_df,
        min_days_required=min_days_required,
        make_min_days_soft=make_min_days_soft,
        allow_borrow=allow_borrow,
        slot_minutes=slot_minutes,
        priority_mode=priority_mode,
        verbose=verbose
    )
    return schedule_micro_df, micro_df, mapping_df


In [4]:
# bays_df: ["Team","BayID","Capacity"]
# subteams_df: ["Team","Subteam","Shift","Size"]

# Strict lexicographic mimic in ONE solve:
sched_micro, micro_df, mapping_df = run_team_total_pipeline_weighted(
    bays_df, subteams_df,
    min_days_required=3,
    make_min_days_soft=True,         # important for weighted attendance
    allow_borrow=False,              # True if cross-team seating allowed
    slot_minutes=30,
    priority_mode="dominance",       # or "normalized" for stable numerics
    verbose=True
)
display(sched_micro)
sched_micro.to_csv("weekly_schedule_final.csv")

Solver status: Optimal


,Mon,Tue,Wed,Thu,Fri,MissedDays
SubteamMicro,,,,,,
A1#1,Remote,Seat_A1(25) + Seat_A2(3),Remote,Remote,Seat_A1(25) + Seat_A2(3),1
A1#2,Seat_A1(4) + Seat_A2(2),Remote,Seat_A1(4) + Seat_A2(2),Seat_A1(6),Remote,0
A2#1,Seat_A1(21),Remote,Seat_A1(21),Seat_A1(19) + Seat_A2(2),Remote,0
B1#1,Remote,Remote,Seat_B1(26),Remote,Remote,2
B2#1,Remote,Remote,Remote,Remote,Seat_B1(23),2
B3#1,Remote,Seat_B2(2),Seat_B1(2),Seat_B1(2),Seat_B2(2),0
B4#1,Seat_B1(19),Seat_B1(19),Remote,Seat_B1(19),Remote,0
C1#1,Seat_C2(63),Seat_C2(63),Seat_C7(63),Remote,Remote,0
C2#1,Seat_C1(55) + Seat_C2(63) + Seat_C3(11) + Seat...,Seat_C1(55) + Seat_C3(11) + Seat_C5(115) + Sea...,Seat_C1(1) + Seat_C2(126) + Seat_C3(11) + Seat...,Remote,Remote,0
